In [1]:
import re
import random
import os

from tqdm import tqdm
import pandas as pd
import torch

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, RobertaTokenizerFast, get_scheduler
from transformers import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, Features, Sequence, Value, ClassLabel
from accelerate import Accelerator

from sklearn.metrics import precision_recall_fscore_support

/home/res/anaconda3/envs/pyenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reduce VRAM usage by reducing fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


## Combine all .txt and .ann files and combine them per medicine

In [3]:
# Base folders containing annotation and text files
annotations_folder = 'annotations/'
original_texts_folder = 'originaltexts/'
output_folder = 'output_datasets/'
os.makedirs(output_folder, exist_ok=True)

# Group files by medicine
file_groups = {}
for file_name in os.listdir(annotations_folder):
    if file_name.endswith('.ann'):
        base_name = '.'.join(file_name.split('.')[:-1])
        medicine = base_name.rsplit('.', 1)[0]
        file_groups.setdefault(medicine, []).append(file_name)

# Process each group
for medicine, ann_files in file_groups.items():
    combined_output = []

    for ann_file in ann_files:
        txt_file = ann_file.replace('.ann', '.txt')
        txt_path = os.path.join(original_texts_folder, txt_file)
        ann_path = os.path.join(annotations_folder, ann_file)

        # Ensure the corresponding .txt file exists
        if not os.path.exists(txt_path):
            raise FileNotFoundError(f"Text file not found for annotation file {ann_file}")

        # Read the content of the .ann and .txt files
        with open(ann_path, 'r') as ann_f:
            ann_lines = ann_f.readlines()

        with open(txt_path, 'r') as txt_f:
            txt_content = txt_f.read()

        # Parse annotations and filter out AnnotatorNotes
        annotations = []
        for line in ann_lines:
            if line.startswith('T'):
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    tag_info, word = parts[1], parts[2]
                    tag_parts = tag_info.split()
                    if len(tag_parts) >= 3:
                        tag = tag_parts[0]
                        try:
                            start_idx = int(tag_parts[1])
                            end_idx = int(tag_parts[2])
                        except ValueError:
                            if ';' in tag_parts[2]:  # Handle ranges like '742;763'
                                start_idx = int(tag_parts[1])
                                end_idx = int(tag_parts[2].split(';')[-1])
                            else:
                                raise ValueError(f"Unexpected annotation format: {tag_parts}")
                        annotations.append((start_idx, end_idx, tag, word))

        # Sort annotations by start index
        annotations.sort(key=lambda x: x[0])

        # Generate output format
        output = []
        current_idx = 0
        for start_idx, end_idx, tag, word in annotations:
            # Add text between the last annotation and the current annotation as "O"
            if current_idx < start_idx:
                intervening_text = txt_content[current_idx:start_idx]
                for token in re.findall(r"\w+(?:'\w+)?|[.,!?]", intervening_text):
                    output.append(f"{token} O")

            # Add the annotated word with its tag
            for i, token in enumerate(word.split()):
                tag_prefix = 'B-' if i == 0 else 'I-'
                output.append(f"{token} {tag_prefix}{tag}")

            current_idx = end_idx

        # Add remaining text as "O"
        if current_idx < len(txt_content):
            remaining_text = txt_content[current_idx:]
            for token in re.findall(r"\w+(?:'\w+)?|[.,!?]", remaining_text):
                output.append(f"{token} O")

        # Add to combined output with a newline separator
        combined_output.extend(output)
        combined_output.append('')  # Empty line between posts

    # Write combined output to file
    combined_output_text = '\n'.join(combined_output).strip()
    output_file = os.path.join(output_folder, f"{medicine}_combined_output.txt")
    with open(output_file, 'w') as out_f:
        out_f.write(combined_output_text)

    print(f"Combined output saved for {medicine} in {output_file}")

Combined output saved for LIPITOR in output_datasets/LIPITOR_combined_output.txt
Combined output saved for VOLTAREN-XR in output_datasets/VOLTAREN-XR_combined_output.txt
Combined output saved for ARTHROTEC in output_datasets/ARTHROTEC_combined_output.txt
Combined output saved for VOLTAREN in output_datasets/VOLTAREN_combined_output.txt
Combined output saved for DICLOFENAC-SODIUM in output_datasets/DICLOFENAC-SODIUM_combined_output.txt
Combined output saved for ZIPSOR in output_datasets/ZIPSOR_combined_output.txt
Combined output saved for FLECTOR in output_datasets/FLECTOR_combined_output.txt
Combined output saved for PENNSAID in output_datasets/PENNSAID_combined_output.txt
Combined output saved for CATAFLAM in output_datasets/CATAFLAM_combined_output.txt
Combined output saved for SOLARAZE in output_datasets/SOLARAZE_combined_output.txt
Combined output saved for DICLOFENAC-POTASSIUM in output_datasets/DICLOFENAC-POTASSIUM_combined_output.txt
Combined output saved for CAMBIA in output_da

## Combine all the medicine files into one dataset

In [5]:
# Folder containing all combined output files
output_datasets_folder = 'output_datasets/'
final_output_file = 'final_dataset.txt'

# Ensure the folder exists
if not os.path.exists(output_datasets_folder):
    raise FileNotFoundError(f"The folder {output_datasets_folder} does not exist.")

# List all files in the folder
output_files = [f for f in os.listdir(output_datasets_folder) if f.endswith('_combined_output.txt')]

# Combine all files into a single final dataset
final_dataset = []
for file_name in output_files:
    file_path = os.path.join(output_datasets_folder, file_name)
    with open(file_path, 'r') as f:
        content = f.read().strip()  # Read and strip any trailing spaces or newlines
        final_dataset.append(content)

    # Add an empty line to separate posts from different files
    final_dataset.append('')

# Write the combined dataset to the final output file
with open(final_output_file, 'w') as f:
    f.write('\n'.join(final_dataset).strip())  # Ensure no extra trailing newline

print(f"Final dataset saved to {final_output_file}")

Final dataset saved to final_dataset.txt


## Read the final dataset into the Iob dataset format

In [6]:
def read_iob_file(file_path):
    """Reads an IOB file from filepath and returns sentences with tokens and tags."""
    sentences = []
    sentence_tokens = []
    sentence_labels = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:  # If line is not empty
                token, tag = line.split()
                sentence_tokens.append(token)
                sentence_labels.append(tag)

            else:
                # End of a sentence
                if sentence_tokens:
                    sentences.append({"tokens": sentence_tokens, "ner_tags": sentence_labels})
                    sentence_tokens = []
                    sentence_labels = []
        # Add the last sentence if file doesn't end with a newline
        if sentence_tokens:
            sentences.append({"tokens": sentence_tokens, "ner_tags": sentence_labels})
    return sentences

def create_dataset_from_final_file(final_file_path):
    """Create a dataset from a single IOB file and return it as a DatasetDict."""

    if not os.path.exists(final_file_path):
        raise FileNotFoundError(f"The file {final_file_path} does not exist.")

    # Parse the file
    data = read_iob_file(final_file_path)

    # Define the label names and ClassLabel feature
    unique_labels = sorted(set(tag for d in data for tag in d["ner_tags"]))
    label_feature = ClassLabel(names=unique_labels)

    # Define the Features schema for Hugging Face datasets
    features = Features({
        'tokens': Sequence(Value("string")),
        'ner_tags': Sequence(label_feature)
    })

    # Convert data into a Dataset
    dataset = Dataset.from_list(data).cast(features)

    # Create a DatasetDict
    dataset_dict = DatasetDict({"full_data": dataset})

    return dataset_dict


In [7]:
final_dataset_path = "final_dataset.txt"
dataset_dict = create_dataset_from_final_file(final_dataset_path)
dataset = dataset_dict['full_data']

Casting the dataset: 100%|██████████| 1248/1248 [00:00<00:00, 13490.75 examples/s]


## Dataset generators

In [8]:
def generate_train_datasets(dataset_, number_of_samples, number_of_splits):
    """
    Generates train datasets by sampling from the given dataset based on the number of samples and splits.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        number_of_samples (int): Number of samples per dataset.
        number_of_splits (int): Number of datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset, List[int]]]: List of generated datasets with their names and indices.
    """
    datasets = []

    for seed in range(number_of_splits):
        # Set the random seed for reproducibility
        random.seed(seed)

        # Shuffle and sample from the dataset
        indices = list(range(len(dataset_)))
        random.shuffle(indices)
        sampled_indices = indices[:number_of_samples]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name and indices
        datasets.append((f"train_dataset_{number_of_samples}_{seed}", sampled_dataset, sampled_indices))

    return datasets

In [9]:
def generate_validation_datasets(dataset_, train_indices, number_of_samples, number_of_splits):
    """
    Generates validation datasets by sampling from the given dataset, ensuring no overlap with training data.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        train_indices (List[int]): Indices of the training dataset to exclude from sampling.
        number_of_samples (int): Number of samples per validation dataset.
        number_of_splits (int): Number of validation datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset, List[int]]]: List of generated validation datasets with names and indices.
    """
    datasets = []
    all_indices = set(range(len(dataset_)))
    available_indices = list(all_indices - set(train_indices))  # Exclude training indices

    for seed in range(number_of_splits):
        # Set the random seed for reproducibility
        random.seed(seed)

        # Shuffle and sample from the remaining indices
        random.shuffle(available_indices)
        sampled_indices = available_indices[:int(number_of_samples / 5)]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name and indices
        datasets.append((f"val_dataset_{number_of_samples/5}_{seed}", sampled_dataset, sampled_indices))

    return datasets

In [10]:
def generate_test_datasets(dataset_, train_indices, val_indices, number_of_samples, number_of_splits):
    """
    Generates test datasets by sampling from the given dataset, ensuring no overlap with training or validation data.

    Args:
        dataset_ (Dataset): The base dataset to sample from.
        train_indices (List[int]): Indices of the training dataset to exclude from sampling.
        val_indices (List[int]): Indices of the validation dataset to exclude from sampling.
        number_of_samples (int): Number of samples per test dataset.
        number_of_splits (int): Number of test datasets to generate (different seeds).

    Returns:
        List[Tuple[str, Dataset]]: List of generated test datasets with names.
    """
    datasets = []
    all_indices = set(range(len(dataset_)))
    available_indices = list(all_indices - set(train_indices) - set(val_indices))  # Exclude train and val indices

    for seed in range(number_of_splits):
        sampled_indices = available_indices[:]

        sampled_dataset = dataset_.select(sampled_indices)

        # Add the dataset with its name
        datasets.append((f"test_dataset_{number_of_samples}_{seed}", sampled_dataset))

    return datasets

### Example usage

In [11]:
# Step 1: Generate Train Dataset
# train_datasets = generate_train_datasets(dataset, number_of_samples=30, number_of_splits=1)
# train_name, train_dataset, train_indices = train_datasets[0]
# print(f"{train_name}: {len(train_dataset)} samples")
#
# # Step 2: Generate Validation Dataset
# val_datasets = generate_validation_datasets(dataset, train_indices, number_of_samples=30, number_of_splits=1)
# val_name, val_dataset, val_indices = val_datasets[0]
# print(f"{val_name}: {len(val_dataset)} samples")
#
# # Step 3: Generate Test Dataset
# test_datasets = generate_test_datasets(dataset, train_indices, val_indices, number_of_samples=30, number_of_splits=1)
# test_name, test_dataset = test_datasets[0]
# print(f"{test_name}: {len(test_dataset)} samples")

# Prepping for training

In [11]:
label_names = dataset.features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [12]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Flatten predictions and labels, removing ignored indices
    true_labels = [label for label_seq in labels for label in label_seq if label != -100]
    true_predictions = [pred for pred_seq, label_seq in zip(predictions, labels)
                        for pred, label in zip(pred_seq, label_seq) if label != -100]
    return true_labels, true_predictions


# Training and evaluation

In [13]:
def iterate_and_finetune_with_torch(
    dataset,
    file_name,
    models,
    start_size=5,
    end_size=500,
    step_size=5,
    k_splits=5,
    batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.0,
    num_epochs=3,
):
    """
    Fine-tune models with varying dataset sizes and k-fold splits, saving results to Excel.

    Parameters:
    - dataset (DatasetDict): Dataset for training, validation, and testing.
    - file_name (str): Excel file to save results.
    - models (dict): Dictionary of model names and their sizes.
    - start_size (int): Starting size for training datasets.
    - end_size (int): Maximum size for training datasets.
    - step_size (int): Step size for increasing dataset sizes.
    - k_splits (int): Number of k-fold splits.
    - batch_size (int): Training batch size.
    - learning_rate (float): Learning rate for fine-tuning.
    - weight_decay (float): Weight decay for optimizer.
    - num_epochs (int): Number of training epochs.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Check or create the results file
    if os.path.exists(file_name):
        results_df = pd.read_excel(file_name)
    else:
        results_df = pd.DataFrame(columns=["Train Size", "K-Fold", "Test F1", "Model"])

    for train_size in range(start_size, end_size + 1, step_size):
        for split in range(k_splits):
            for size, model_name in models.items():
                print(f"\nFine-tuning {model_name} ({size}) with Train Size {train_size}, Split {split + 1}...")

                # Initialize tokenizer
                if size == "large":
                    tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large", add_prefix_space=True)
                else:
                    tokenizer = AutoTokenizer.from_pretrained(model_name)

                data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

                # Generate datasets
                train_datasets = generate_train_datasets(
                    dataset, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, train_dataset, train_indices = train_datasets[split]

                val_datasets = generate_validation_datasets(
                    dataset, train_indices=train_indices, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, val_dataset, val_indices = val_datasets[split]

                test_datasets = generate_test_datasets(
                    dataset, train_indices=train_indices, val_indices=val_indices,
                    number_of_samples=train_size, number_of_splits=k_splits
                )
                _, test_dataset = test_datasets[split]

                def align_labels_with_tokens(labels_, word_ids):
                    new_labels = []
                    current_word = None

                    for word_id in word_ids:
                        if word_id != current_word:
                            current_word = word_id
                            label = -100 if word_id is None else labels_[word_id]
                            new_labels.append(label)

                        elif word_id is None:
                            # Special token
                            new_labels.append(-100)

                        else:
                            # Same word as previous token
                            label = labels_[word_id]

                            # If the label is B-XXX we change it to I-XXX
                            if label % 2 == 1:
                                label += 1
                            new_labels.append(label)

                    return new_labels


                def tokenize_and_align_labels(examples):
                    tokenized_inputs = tokenizer(
                        examples["tokens"], truncation=True,
                        is_split_into_words=True
                    )
                    all_labels = examples["ner_tags"]
                    new_labels = []
                    for i, labels_ in enumerate(all_labels):
                        word_ids = tokenized_inputs.word_ids(i)
                        new_labels.append(align_labels_with_tokens(labels_, word_ids))

                    tokenized_inputs["labels"] = new_labels
                    return tokenized_inputs

                # Tokenize datasets
                tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)

                train_dataloader = DataLoader(tokenized_train, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
                val_dataloader = DataLoader(tokenized_val, batch_size=batch_size, collate_fn=data_collator)
                test_dataloader = DataLoader(tokenized_test, batch_size=batch_size, collate_fn=data_collator)

                # Initialize the model for token classification
                model = AutoModelForTokenClassification.from_pretrained(
                    model_name, id2label=id2label, label2id=label2id
                )
                
                #Optimize GPU ram at cost of some speed
                model.gradient_checkpointing_enable()


                optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                num_training_steps = num_epochs * len(train_dataloader)
                lr_scheduler = get_scheduler(
                    "linear",
                    optimizer=optimizer,
                    num_warmup_steps=0,
                    num_training_steps=num_training_steps
                )

                accelerator = Accelerator()
                model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(
                    model, optimizer, train_dataloader, val_dataloader
                )


                # Training loop
                for epoch in range(num_epochs):
                    print(f"Epoch {epoch + 1}/{num_epochs}")
                    model.train()
                    total_loss = 0
                    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}")
                    for batch in progress_bar:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        loss = outputs.loss
                        total_loss += loss.item()

                        accelerator.backward(loss)
                        optimizer.step()
                        lr_scheduler.step()
                        optimizer.zero_grad()
                        progress_bar.set_postfix(loss=loss.item())

                    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")

                # Validation loop
                model.eval()
                val_predictions, val_labels = [], []
                with torch.no_grad():
                    for batch in val_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        val_labels.extend(flat_labels)
                        val_predictions.extend(flat_predictions)

                # Test loop
                test_predictions, test_labels = [], []
                with torch.no_grad():
                    for batch in test_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        test_labels.extend(flat_labels)
                        test_predictions.extend(flat_predictions)

                # Calculate test metrics
                precision, recall, f1, _ = precision_recall_fscore_support(
                    test_labels, test_predictions, average="micro"
                )
                print(f"Test Metrics: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

                # Append results
                new_row = pd.DataFrame(
                    [{"Train Size": train_size, "K-Fold": split + 1, "Test F1": f1, "Model": model_name}]
                )
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                results_df.to_excel(file_name, index=False)

                # Cleanup
                del model, train_dataloader, val_dataloader, test_dataloader, predictions, labels, test_labels
                del test_predictions, logits, loss, outputs, precision, recall, f1, new_row 
                del val_dataset, train_dataset, test_dataset, val_datasets, train_datasets, test_datasets
                del optimizer, tokenized_train, tokenized_test, tokenized_val, tokenizer
                torch.cuda.empty_cache()

    print(f"Results saved to {file_name}")


In [16]:
models = {
    "small": "bert-base-cased",
    "medium": "bert-large-cased",
    "large": "roberta-large"
}

iterate_and_finetune_with_torch(dataset=dataset, file_name='Experiments_full_labeled.xlsx', models=models, start_size=500, end_size=500, step_size=5)


Fine-tuning bert-base-cased (small) with Train Size 500, Split 1...


Map: 100%|██████████| 648/648 [00:00<00:00, 9405.15 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training Epoch 1: 100%|██████████| 63/63 [00:10<00:00,  6.28it/s, loss=0.4]  


Epoch 1 Loss: 36.1593
Epoch 2/3


...

Epoch 1 Loss: 36.3731
Epoch 2/3


Training Epoch 2: 100%|██████████| 63/63 [00:31<00:00,  2.01it/s, loss=0.176]


Epoch 2 Loss: 17.1399
Epoch 3/3


Training Epoch 3: 100%|██████████| 63/63 [00:31<00:00,  1.99it/s, loss=0.0931]


Epoch 3 Loss: 12.5362
Test Metrics: Precision=0.9319, Recall=0.9319, F1=0.9319
Results saved to Experiments_full_labeled.xlsx


In [15]:
models = {
    "large": "roberta-large"
}

iterate_and_finetune_with_torch(dataset=dataset, file_name='Experiments_epoch4_roberta.xlsx', models=models, start_size=230, end_size=500, step_size=5, num_epochs=4)


Fine-tuning roberta-large (large) with Train Size 230, Split 1...


Map: 100%|██████████| 972/972 [00:00<00:00, 9899.70 examples/s]
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4


Training Epoch 1: 100%|██████████| 29/29 [00:12<00:00,  2.32it/s, loss=0.451]


Epoch 1 Loss: 17.1539
Epoch 2/4


...

Epoch 2 Loss: 15.6231
Epoch 3/4


Training Epoch 3: 100%|██████████| 63/63 [00:29<00:00,  2.12it/s, loss=0.196] 


Epoch 3 Loss: 10.5744
Epoch 4/4


Training Epoch 4: 100%|██████████| 63/63 [00:31<00:00,  2.01it/s, loss=0.0924]


Epoch 4 Loss: 7.6449
Test Metrics: Precision=0.9372, Recall=0.9372, F1=0.9372
Results saved to Experiments_epoch4_roberta.xlsx


In [14]:
models = {
    "large": "roberta-large"
}

iterate_and_finetune_with_torch(dataset=dataset, file_name='Experiments_epoch5_roberta.xlsx', models=models, start_size=210, end_size=500, step_size=5, num_epochs=5)


Fine-tuning roberta-large (large) with Train Size 210, Split 1...


Map: 100%|██████████| 996/996 [00:00<00:00, 9574.06 examples/s]
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


Training Epoch 1: 100%|██████████| 27/27 [00:11<00:00,  2.35it/s, loss=0.356]


Epoch 1 Loss: 17.5578
Epoch 2/5


...

Epoch 3 Loss: 9.2853
Epoch 4/5


Training Epoch 4: 100%|██████████| 63/63 [00:31<00:00,  2.00it/s, loss=0.0735]


Epoch 4 Loss: 6.2381
Epoch 5/5


Training Epoch 5: 100%|██████████| 63/63 [00:30<00:00,  2.04it/s, loss=0.0638]


Epoch 5 Loss: 4.3945
Test Metrics: Precision=0.9398, Recall=0.9398, F1=0.9398
Results saved to Experiments_epoch5_roberta.xlsx


In [15]:
models = {
    "small": "bert-base-cased",
    "medium": "bert-large-cased",
    "large": "roberta-large"
}

iterate_and_finetune_with_torch(dataset=dataset, file_name='Experiments_moreksplits10_lesssteps20_for_smoother_graphh.xlsx', models=models, start_size=360, end_size=500, step_size=20, k_splits=10)


Fine-tuning bert-base-cased (small) with Train Size 360, Split 1...


Map: 100%|██████████| 816/816 [00:00<00:00, 9336.01 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training Epoch 1: 100%|██████████| 45/45 [00:07<00:00,  6.01it/s, loss=0.439]


Epoch 1 Loss: 31.2289
Epoch 2/3


...

Epoch 1 Loss: 28.7331
Epoch 2/3


Training Epoch 2: 100%|██████████| 63/63 [00:30<00:00,  2.05it/s, loss=0.271]


Epoch 2 Loss: 13.5108
Epoch 3/3


Training Epoch 3: 100%|██████████| 63/63 [00:30<00:00,  2.07it/s, loss=0.122] 


Epoch 3 Loss: 8.9374
Test Metrics: Precision=0.9363, Recall=0.9363, F1=0.9363
Results saved to Experiments_moreksplits10_lesssteps20_for_smoother_graphh.xlsx


### Lipitor Analysis

In [14]:
lipitor_dataset_path = "output_datasets/LIPITOR_combined_output.txt"
dataset_dict_lip = create_dataset_from_final_file(lipitor_dataset_path)
dataset_lip = dataset_dict_lip['full_data']

label_names_lip = dataset_lip.features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(label_names_lip)}
label2id = {v: k for k, v in id2label.items()}

Casting the dataset: 100%|██████████| 999/999 [00:00<00:00, 24829.69 examples/s]


In [15]:
def iterate_and_finetune_with_torch_lip(
    dataset,
    file_name,
    models,
    start_size=5,
    end_size=500,
    step_size=5,
    k_splits=5,
    batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.0,
    num_epochs=3,
):
    """
    Fine-tune models with varying dataset sizes and k-fold splits, saving results to Excel.

    Parameters:
    - dataset (DatasetDict): Dataset for training, validation, and testing.
    - file_name (str): Excel file to save results.
    - models (dict): Dictionary of model names and their sizes.
    - start_size (int): Starting size for training datasets.
    - end_size (int): Maximum size for training datasets.
    - step_size (int): Step size for increasing dataset sizes.
    - k_splits (int): Number of k-fold splits.
    - batch_size (int): Training batch size.
    - learning_rate (float): Learning rate for fine-tuning.
    - weight_decay (float): Weight decay for optimizer.
    - num_epochs (int): Number of training epochs.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Check or create the results file
    if os.path.exists(file_name):
        results_df = pd.read_excel(file_name)
    else:
        results_df = pd.DataFrame(columns=["Train Size", "K-Fold", "Test F1", "Model"])

    for train_size in range(start_size, end_size + 1, step_size):
        for split in range(k_splits):
            for size, model_name in models.items():
                print(f"\nFine-tuning {model_name} ({size}) with Train Size {train_size}, Split {split + 1}...")

                # Initialize tokenizer
                if size == "large":
                    tokenizer = RobertaTokenizerFast.from_pretrained("roberta-large", add_prefix_space=True)
                else:
                    tokenizer = AutoTokenizer.from_pretrained(model_name)

                data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

                # Generate datasets
                train_datasets = generate_train_datasets(
                    dataset, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, train_dataset, train_indices = train_datasets[split]

                val_datasets = generate_validation_datasets(
                    dataset, train_indices=train_indices, number_of_samples=train_size, number_of_splits=k_splits
                )
                _, val_dataset, val_indices = val_datasets[split]

                test_datasets = generate_test_datasets(
                    dataset, train_indices=train_indices, val_indices=val_indices,
                    number_of_samples=train_size, number_of_splits=k_splits
                )
                _, test_dataset = test_datasets[split]

                def align_labels_with_tokens(labels_, word_ids):
                    new_labels = []
                    current_word = None

                    for word_id in word_ids:
                        if word_id != current_word:
                            current_word = word_id
                            label = -100 if word_id is None else labels_[word_id]
                            new_labels.append(label)

                        elif word_id is None:
                            # Special token
                            new_labels.append(-100)

                        else:
                            # Same word as previous token
                            label = labels_[word_id]

                            # If the label is B-XXX we change it to I-XXX
                            if label % 2 == 1:
                                label += 1
                            new_labels.append(label)

                    return new_labels


                def tokenize_and_align_labels(examples):
                    tokenized_inputs = tokenizer(
                        examples["tokens"], truncation=True,
                        is_split_into_words=True
                    )
                    all_labels = examples["ner_tags"]
                    new_labels = []
                    for i, labels_ in enumerate(all_labels):
                        word_ids = tokenized_inputs.word_ids(i)
                        new_labels.append(align_labels_with_tokens(labels_, word_ids))

                    tokenized_inputs["labels"] = new_labels
                    return tokenized_inputs

                # Tokenize datasets
                tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)
                tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True,  remove_columns=dataset.column_names)

                train_dataloader = DataLoader(tokenized_train, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
                val_dataloader = DataLoader(tokenized_val, batch_size=batch_size, collate_fn=data_collator)
                test_dataloader = DataLoader(tokenized_test, batch_size=batch_size, collate_fn=data_collator)

                # Initialize the model for token classification
                model = AutoModelForTokenClassification.from_pretrained(
                    model_name, id2label=id2label, label2id=label2id
                )
                
                #Optimize GPU ram at cost of some speed
                model.gradient_checkpointing_enable()


                optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                num_training_steps = num_epochs * len(train_dataloader)
                lr_scheduler = get_scheduler(
                    "linear",
                    optimizer=optimizer,
                    num_warmup_steps=0,
                    num_training_steps=num_training_steps
                )

                accelerator = Accelerator()
                model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(
                    model, optimizer, train_dataloader, val_dataloader
                )


                # Training loop
                for epoch in range(num_epochs):
                    print(f"Epoch {epoch + 1}/{num_epochs}")
                    model.train()
                    total_loss = 0
                    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}")
                    for batch in progress_bar:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        loss = outputs.loss
                        total_loss += loss.item()

                        accelerator.backward(loss)
                        optimizer.step()
                        lr_scheduler.step()
                        optimizer.zero_grad()
                        progress_bar.set_postfix(loss=loss.item())

                    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")

                # Validation loop
                model.eval()
                val_predictions, val_labels = [], []
                with torch.no_grad():
                    for batch in val_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        val_labels.extend(flat_labels)
                        val_predictions.extend(flat_predictions)

                # Test loop
                test_predictions, test_labels = [], []
                with torch.no_grad():
                    for batch in test_dataloader:
                        batch = {k: v.to(device) for k, v in batch.items()}
                        outputs = model(**batch)
                        logits = outputs.logits
                        predictions = logits.argmax(dim=-1)
                        labels = batch["labels"]

                        predictions = accelerator.gather(predictions)
                        labels = accelerator.gather(labels)

                        flat_labels, flat_predictions = postprocess(predictions, labels)
                        test_labels.extend(flat_labels)
                        test_predictions.extend(flat_predictions)

                # Calculate test metrics
                precision, recall, f1, _ = precision_recall_fscore_support(
                    test_labels, test_predictions, average="micro"
                )
                print(f"Test Metrics: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

                # Append results
                new_row = pd.DataFrame(
                    [{"Train Size": train_size, "K-Fold": split + 1, "Test F1": f1, "Model": model_name}]
                )
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                results_df.to_excel(file_name, index=False)

                # Cleanup
                del model, train_dataloader, val_dataloader, test_dataloader, predictions, labels, test_labels
                del test_predictions, logits, loss, outputs, precision, recall, f1, new_row 
                del val_dataset, train_dataset, test_dataset, val_datasets, train_datasets, test_datasets
                del optimizer, tokenized_train, tokenized_test, tokenized_val, tokenizer
                torch.cuda.empty_cache()

    print(f"Results saved to {file_name}")


In [ ]:
# Define the models and their corresponding sizes
models = {
    "small": "bert-base-cased",
    "medium": "bert-large-cased",
    "large": "roberta-large"
}

iterate_and_finetune_with_torch_lip(dataset=dataset_lip, file_name='Experiments_lipitor.xlsx', models=models, start_size=5, end_size=500, step_size=5)


Fine-tuning bert-base-cased (small) with Train Size 5, Split 1...


Map: 100%|██████████| 993/993 [00:00<00:00, 8831.33 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training Epoch 1: 100%|██████████| 1/1 [00:00<00:00,  3.91it/s, loss=2.73]


Epoch 1 Loss: 2.7308
Epoch 2/3


...

Map: 100%|██████████| 807/807 [00:00<00:00, 9030.36 examples/s]
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Training Epoch 1: 100%|██████████| 20/20 [00:10<00:00,  1.86it/s, loss=0.34] 


Epoch 1 Loss: 14.8218
Epoch 2/3


Training Epoch 2:  95%|█████████▌| 19/20 [00:08<00:00,  2.28it/s, loss=0.396]